In [1]:
!pip install crewai crewai_tools arxiv

In [2]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [29]:
# Set up environment variables
import os

groq_api_key = ""
os.environ["GROQ_API_KEY"] = groq_api_key
os.environ["GROQ_MODEL_NAME"] = "qwen-qwq-32b"

In [27]:
from crewai import Agent, Task, Crew, LLM

# Creating Arxiv Search Tool

In [24]:
from typing import Type, List
from pydantic import BaseModel
from crewai.tools import BaseTool
import arxiv
import time

class FetchArxivTyphoonAIPapersTool(BaseTool):
    name: str = "fetch_arxiv_papers"
    description: str = "Fetches the newest arXiv papers related to Typhoon in the AI field."

    def _run(self) -> List[dict]:
        # Define the search query for Typhoon in AI field
        query = "all:typhoon AND cat:cs.AI"  # Search for 'typhoon' in AI category

        # Initialize the arXiv client
        client = arxiv.Client(
            page_size=10,  # Fetch 10 results per page
            delay_seconds=3  # Delay between requests to respect rate limits
        )

        # Create a search
        search = arxiv.Search(
            query=query,
            sort_by=arxiv.SortCriterion.SubmittedDate,  # Sort by submission date (newest first)
            sort_order=arxiv.SortOrder.Descending,      # Descending order for newest first
            max_results=10  # Limit to 10 results
        )

        print(f"Searching arXiv for AI papers about Typhoon")

        # Collect results
        all_papers = []
        for result in client.results(search):
            all_papers.append({
                'title': result.title,
                'authors': [author.name for author in result.authors],
                'summary': result.summary,
                'published': result.published,
                'url': result.entry_id
            })

            # Delay between requests to respect rate limits
            time.sleep(3)

            # Stop once we have 10 papers
            if len(all_papers) >= 10:
                break

        print(f"Fetched {len(all_papers)} AI papers about Typhoon")
        return all_papers

arxiv_search_tool = FetchArxivTyphoonAIPapersTool()

In [25]:
llm = LLM(
    model=f"groq/{os.environ['GROQ_MODEL_NAME']}",
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1",
)

# Agent 1: AI Typhoon Researcher
researcher = Agent(
    role = "Senior AI Researcher",
    goal = "Find the top 3 AI papers related to Typhoon from the search results from arXiv. Rank them based on relevance and technical significance.",
    backstory = "You are a senior researcher specializing in artificial intelligence with knowledge of how AI is applied to typhoon prediction, monitoring, and response. You can identify the most significant AI research related to typhoons based on title and abstract.",
    verbose = True,
    tools = [arxiv_search_tool],
    llm=llm
)

# Agent 2: Frontend Engineer
frontend_engineer = Agent(
    role = "Senior Frontend & AI Engineer",
    goal = "Compile the AI typhoon research results into an informative HTML report.",
    backstory = "You are a competent frontend engineer with decades of experience in HTML and CSS. You specialize in presenting complex AI research and applications in accessible formats.",
    verbose = True,
    llm=llm
)

# Task for AI Typhoon Researcher
research_task = Task(
    description = ("Find the top 3 newest AI research papers related to Typhoons from arXiv."),
    expected_output = (
        "A list of top 3 AI papers about typhoons with the following information in the following format:"
        "- Title"
        "- Authors"
        "- Abstract"
        "- Link to the paper"
        "- Publication date"
    ),
    agent = researcher,
    human_input = True,
)

# Task for Frontend Engineer
reporting_task = Task(
    description = ("Compile the AI typhoon research results into a detailed report in an HTML file."),
    expected_output = (
        "An HTML file with the results in the following format:"
        "Top 3 AI Research Papers on Typhoons"
        "Use the tabular format for the following:"
        "- Title (which on clicking opens the paper in a new tab)"
        "- Authors"
        "- Publication date"
        "- Short summary of the abstract (2-4 sentences)"
        "Please do not add '''html''' to the top and bottom of the final file."
    ),
    agent = frontend_engineer,
    context = [research_task],
    output_file = "./ai_typhoon_research_report.html",
    human_input = True,
)

ai_typhoon_research_crew = Crew(
    agents = [researcher, frontend_engineer],
    tasks = [research_task, reporting_task],
    verbose = True,
)

result = ai_typhoon_research_crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: dc46478b-7b3e-4a37-a0be-129356022972                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 072a6a80-4868-4018-bb16-629e23852241
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 072a6a80-4868-4018-bb16-629e23852241
       Status: Executing Task...
    └── 🤖 Agent: Senior AI Researcher
            Status: In Progress

# Agent: Senior AI Researcher
## Task: Find the top 3 newest AI research papers related to Typhoons from arXiv.


🚀 Crew: crew
└── 📋 Task: 072a6a80-4868-4018-bb16-629e23852241
       Status: Executing Task...
    └── 🤖 Agent: Senior AI Researcher
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Senior Frontend & AI Engineer
## Final Answer: 
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI Research on Typhoons</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            line-height: 1.6;
            margin: 40px;
            color: #333;
        }
        h1 {
            color: #2c3e50;
            text-align: center;
        }
        table {
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
        }
        th, td {
            border: 1px solid #ddd;
            padding: 12px;
            text-align: left;
        }
        th {
            background-color: #f8f9fa;
            font-weight: bold;
        }
        .paper-title {
            color: #3498db;
            text-decoration: none;
        }
        .paper-title:hover {
            text-decoration: underline;
       

🚀 Crew: crew
├── 📋 Task: 072a6a80-4868-4018-bb16-629e23852241
│      Assigned to: Senior AI Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior AI Researcher
│           Status: ✅ Completed
└── 📋 Task: 2c898630-0994-424d-b427-5c7cdf9ff354
       Status: Executing Task...
    └── 🤖 Agent: Senior Frontend & AI Engineer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 072a6a80-4868-4018-bb16-629e23852241
│      Assigned to: Senior AI Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior AI Researcher
│           Status: ✅ Completed
└── 📋 Task: 2c898630-0994-424d-b427-5c7cdf9ff354
       Assigned to: Senior Frontend & AI Engineer
       Status: ✅ Completed
    └── 🤖 Agent: Senior Frontend & AI Engineer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 2c898630-0994-424d-b427-5c7cdf9ff354                                                                     │
│  Agent: Senior Frontend & AI Engineer                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: dc46478b-7b3e-4a37-a0be-129356022972                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [28]:
from typing import Type, List
from pydantic import BaseModel
from crewai.tools import BaseTool
import arxiv
import time

class FetchArxivTyphoonLLMPapersTool(BaseTool):
    name: str = "fetch_arxiv_papers"
    description: str = "Fetches arXiv papers related to the 'Typhoon' language model in Computation and Language (cs.CL) field."

    def _run(self) -> List[dict]:
        # Define the search query for Typhoon language model in Computation and Language
        query = "all:typhoon AND cat:cs.CL"  # Search for 'typhoon' in Computation and Language category

        # Initialize the arXiv client
        client = arxiv.Client(
            page_size=10,  # Fetch 10 results per page
            delay_seconds=3  # Delay between requests to respect rate limits
        )

        # Create a search
        search = arxiv.Search(
            query=query,
            sort_by=arxiv.SortCriterion.SubmittedDate,  # Sort by submission date (newest first)
            sort_order=arxiv.SortOrder.Descending,      # Descending order for newest first
            max_results=10  # Limit to 10 results
        )

        print(f"Searching arXiv for papers about 'Typhoon' language model in Computation and Language field")

        # Collect results
        all_papers = []
        for result in client.results(search):
            all_papers.append({
                'title': result.title,
                'authors': [author.name for author in result.authors],
                'summary': result.summary,
                'published': result.published,
                'url': result.entry_id,
                'categories': result.categories,
                'comment': result.comment if hasattr(result, 'comment') else None,
                'doi': result.doi if hasattr(result, 'doi') else None
            })

            # Delay between requests to respect rate limits
            time.sleep(3)

            # Stop once we have 10 papers
            if len(all_papers) >= 10:
                break

        print(f"Fetched {len(all_papers)} papers about 'Typhoon' language model")
        return all_papers

arxiv_search_tool = FetchArxivTyphoonLLMPapersTool()

In [30]:
llm = LLM(
    model=f"groq/{os.environ['GROQ_MODEL_NAME']}",
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1",
)

# Agent 1: Typhoon LLM Researcher
researcher = Agent(
    role = "NLP Researcher",
    goal = "Find research papers specifically about the 'Typhoon' Thai language model from arXiv. Focus on papers in the Computation and Language (cs.CL) field.",
    backstory = "You are an NLP researcher specializing in large language models with particular interest in models for low-resource languages. You're looking for papers specifically about the 'Typhoon' Thai language model, not general typhoon weather phenomena.",
    verbose = True,
    tools = [arxiv_search_tool],
    llm=llm
)

# Agent 2: Frontend Engineer
frontend_engineer = Agent(
    role = "Senior Frontend & NLP Engineer",
    goal = "Compile the research results about the 'Typhoon' Thai language model into a detailed HTML report.",
    backstory = "You are a frontend engineer who specializes in presenting NLP research in accessible formats. You understand the importance of language models for low-resource languages and can effectively communicate technical details about them.",
    verbose = True,
    llm=llm
)

# Task for Typhoon LLM Researcher
research_task = Task(
    description = (
        "Search for research papers specifically about the 'Typhoon' Thai language model in the Computation and Language (cs.CL) field. "
        "The paper should be about the actual 'Typhoon' language model for Thai language, not about weather-related typhoons. "
        "Collect comprehensive information about each paper, including authors, submission dates, categories, and DOIs if available."
    ),
    expected_output = (
        "A detailed list of research papers about the 'Typhoon' Thai language model with the following information:"
        "- Title"
        "- Authors"
        "- Abstract"
        "- arXiv URL"
        "- Publication date"
        "- Categories/Subject areas"
        "- Comments (if available)"
        "- DOI (if available)"
    ),
    agent = researcher,
    human_input = True,
)

# Task for Frontend Engineer
reporting_task = Task(
    description = (
        "Create a comprehensive HTML report about research papers on the 'Typhoon' Thai language model. "
        "Format the information in a way that highlights key details about this specific language model. "
        "Include all metadata about each paper to provide a complete research overview."
    ),
    expected_output = (
        "An HTML file with the results formatted as:"
        "Research Papers on 'Typhoon' Thai Language Model"
        "Use an academic-style layout including:"
        "- Title (which on clicking opens the paper in a new tab)"
        "- Authors with proper formatting"
        "- Publication date"
        "- Categories/Subject areas"
        "- DOI with link (if available)"
        "- Concise but comprehensive abstract summary"
        "- Comment section (if available)"
        "Please include proper formatting that would be appropriate for an academic research portal."
    ),
    agent = frontend_engineer,
    context = [research_task],
    output_file = "./typhoon_llm_research_report.html",
    human_input = True,
)

typhoon_llm_research_crew = Crew(
    agents = [researcher, frontend_engineer],
    tasks = [research_task, reporting_task],
    verbose = True,
)

result = typhoon_llm_research_crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: b8e90c11-5cc3-44f6-846b-fb30c5187b5e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 5257b7a5-24bc-42f3-9a7e-7f2c09003ec4
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 5257b7a5-24bc-42f3-9a7e-7f2c09003ec4
       Status: Executing Task...
    └── 🤖 Agent: NLP Researcher
            Status: In Progress

# Agent: NLP Researcher
## Task: Search for research papers specifically about the 'Typhoon' Thai language model in the Computation and Language (cs.CL) field. The paper should be about the actual 'Typhoon' language model for Thai language, not about weather-related typhoons. Collect comprehensive information about each paper, including authors, submission dates, categories, and DOIs if available.


🚀 Crew: crew
└── 📋 Task: 5257b7a5-24bc-42f3-9a7e-7f2c09003ec4
       Status: Executing Task...
    └── 🤖 Agent: NLP Researcher
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Senior Frontend & NLP Engineer
## Final Answer: 
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Research Papers on 'Typhoon' Thai Language Model</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 20px;
            padding: 0;
            line-height: 1.6;
        }
        .container {
            max-width: 1200px;
            margin: auto;
        }
        .paper-container {
            border: 1px solid #ddd;
            padding: 20px;
            margin-bottom: 20px;
            border-radius: 5px;
        }
        .paper-title a {
            text-decoration: none;
            color: #1A0DAB;
            font-weight: bold;
        }
        .paper-title a:hover {
            text-decoration: underline;
        }
        .paper-metadata {
            margin: 10px 0;
        }
        .paper-abstract,
        .paper-comments {
            margin: 15px 0;
        }
        .paper-doi a

🚀 Crew: crew
├── 📋 Task: 5257b7a5-24bc-42f3-9a7e-7f2c09003ec4
│      Assigned to: NLP Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: NLP Researcher
│           Status: ✅ Completed
└── 📋 Task: 2428331a-b623-4936-a8fa-346d3424530f
       Status: Executing Task...
    └── 🤖 Agent: Senior Frontend & NLP Engineer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 5257b7a5-24bc-42f3-9a7e-7f2c09003ec4
│      Assigned to: NLP Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: NLP Researcher
│           Status: ✅ Completed
└── 📋 Task: 2428331a-b623-4936-a8fa-346d3424530f
       Assigned to: Senior Frontend & NLP Engineer
       Status: ✅ Completed
    └── 🤖 Agent: Senior Frontend & NLP Engineer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 2428331a-b623-4936-a8fa-346d3424530f                                                                     │
│  Agent: Senior Frontend & NLP Engineer                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: b8e90c11-5cc3-44f6-846b-fb30c5187b5e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯